In [ ]:
!pip install efficientnet_pytorch
!pip install neptune-client

In [ ]:
import os
import time
import random
import yaml

import numpy as np
import cv2 as cv
import pandas as pd
import neptune.new as neptune
import matplotlib.pyplot as plt

import efficientnet_pytorch

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from pathlib import Path
from tqdm import tqdm

from albumentations import (
    Compose, Normalize, Transpose, HorizontalFlip,
    VerticalFlip, RandomRotate90, RandomScale
)
from albumentations.pytorch import ToTensorV2

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
base_dir = Path('/kaggle/input/cassava-leaf-disease-classification')
custom_data_base_dir = Path('/kaggle/input/cassava-dataset-splitted')
train_img_dir = f'{base_dir}/train_images'
train_df = pd.read_csv(f'{custom_data_base_dir}/cassava_splitted.csv')

In [ ]:
train_df.head()

In [ ]:
class Config:
    cfg = {
        'seed': 42,
        'validation_fold': 0,
        'num_classes': 5,
        'image_size': (512, 512),
        'weight_decay': 1e-6,
        'batch_size': 32,
        'learning_rate': 0.001,
        'criterion': 'cross_entropy',
        'warm_restarts_T_0': 10,
        'warm_restarts_eta_min': 1e-6,
        'epochs': 10,
        'patience': 10,
        'num_workers': 8
        
    }
    
    neptune_settings = {
        'active': True,
        'log_artifacts': False
    }

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(Config.cfg['seed'])

In [ ]:
def get_neptune_config(config_path):
    with open(config_path) as f:
        neptune_config = yaml.load(f, Loader=yaml.FullLoader)
    return neptune_config

In [ ]:
neptune_config = get_neptune_config('/kaggle/input/neptune-configuration-file/neptune_config.yaml')

In [ ]:
if Config.neptune_settings['active']:
    neptune_run = neptune.init(
        project = neptune_config['project'],
        api_token = neptune_config['api_token']
    )
    
    neptune_run['my_params'] = Config.cfg

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, image_size, augments=None):
        self.df = df.image_id.tolist()
        self.targets = df['label'].tolist()
        self.image_size = image_size
        self.augments = augments
        
    def __getitem__(self, idx):
        image = cv.imread(os.path.join(train_img_dir, self.df[idx]))
        image = cv.resize(image, self.image_size)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        if self.augments:
            image = self.augments(image=image)['image']
            
        y = torch.tensor(self.targets[idx], dtype=torch.long)
        
        return {'X': image, 'y': y}
    
    def __len__(self):
        return len(self.targets)

In [ ]:
class Augments:
    train_augments = Compose([
        Transpose(p=.5),
        HorizontalFlip(p=.5),
        VerticalFlip(p=.5),
        RandomRotate90(),
        Normalize(mean=[.485, .456, .406],
                  std=[.229, .224, .225],
                  p=1.),
        ToTensorV2(p=1.),
    ],
    p=1.,
    )
    
    val_augments = Compose([
        Normalize(mean=[.485, .456, .406],
                  std=[.229, .224, .225],
                  p=1.),
        ToTensorV2(p=1.),
    ],
    p=1.,
    )

In [ ]:
def efficientnet_b0(num_classes):
    model = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b0')
    model._fc = nn.Linear(in_features=1280,
                          out_features=num_classes,
                          bias=True)
    return model

In [ ]:
def efficientnet_b4(num_classes):
    model = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b4')
    model._fc = nn.Linear(in_features=1792,
                          out_features=num_classes,
                          bias=True)
    return model

In [ ]:
def resnet_18(num_classes):
    model = torch.hub.load('pytorch/vision:v0.10.1', 'resnet18', pretrained=True)
    model.classifier = nn.Linear(in_features=512,
                                 out_features=num_classes,
                                 bias=True)
    return model

In [ ]:
def resnet_101(num_classes):
    model = torch.hub.load('pytorch/vision:v0.10.1', 'resnet101', pretrained=True)
    model.classifier = nn.Linear(in_features=2048,
                                 out_features=num_classes,
                                 bias=True)
    return model

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, criterion, loss_metric, score_metric, scheduler, device='cuda:0'):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_metric = loss_metric
        self.score_metric = score_metric
        self.scheduler = scheduler
        self.device = device
        
        self.best_valid_score = -np.inf
        self.n_patience = 0
        
        self.messages = {
            "batch": "[{}: {}/{}] loss: {:.5f}, score: {:.5f}, time: {} s",
            "epoch": "[Epoch {}: {}] loss: {:.5f}, score: {:.5f}, time: {} s",
            "checkpoint": "The score improved from {:.5f} to {:.5f}. Save model to '{}'",
            "patience": "\nValid score didn't improve last {} epochs."
        }
        
    
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):
        history = {
            "train_loss": [],
            "train_score": [],
            "valid_loss": [],
            "valid_score": [],
        }
        
        for num_epoch in range(1, epochs + 1):
            self.info_message(f'Epoch: {num_epoch}')
            
            if Config.neptune_settings['active']:
                neptune_run['lr'].log(self.optimizer.param_groups[0]['lr'])
            
            train_loss, train_score, train_time = self.train_epoch(train_loader, num_epoch)
            valid_loss, valid_score, valid_time = self.valid_epoch(valid_loader)
            
            history["train_loss"].append(train_loss)
            history["train_score"].append(train_score)
            history["valid_loss"].append(valid_loss)
            history["valid_score"].append(valid_score)
            
            self.info_message(
                self.messages['epoch'], 'Train', num_epoch, train_loss, train_score, train_time
            )
            
            if Config.neptune_settings['active']:
                neptune_run['train_loss'].log(train_loss)
                neptune_run['train_accuracy'].log(train_score)
            
            self.info_message(
                self.messages['epoch'], 'Train', num_epoch, valid_loss, valid_score, valid_time
            )
            
            if Config.neptune_settings['active']:
                neptune_run['valid_loss'].log(valid_loss)
                neptune_run['valid_accuracy'].log(valid_score)
            
            if self.best_valid_score < valid_score:
                self.info_message(
                    self.messages['checkpoint'], self.best_valid_score, valid_score, save_path
                )
                self.best_valid_score = valid_score
                self.save_model(num_epoch, save_path)
                self.n_patience = 0
            else:
                self.n_patience += 1
                
            if self.n_patience >= patience:
                self.info_message(self.messages['patience'], patience)
                break
        
        return history
    
    def train_epoch(self, train_loader, epoch):
        self.model.train()
        t = time.time()
        train_loss = self.loss_metric()
        train_score = self.score_metric()
        
        for step, batch in enumerate(train_loader, 1):
            images = batch['X'].to(self.device)
            targets = batch['y'].to(self.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(images)
            
            loss = self.criterion(outputs, targets)
            loss.backward()
            
            train_loss.update(loss.detach().item())
            train_score.update(targets, outputs.detach())
            
            self.optimizer.step()
            
            self.info_message(
                self.messages['batch'], 'Train', step, len(train_loader),
                train_loss.avg, train_score.avg, int(time.time() - t), end='\r'
            )
            
        self.scheduler.step()
        
        return train_loss.avg, train_score.avg, int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        valid_loss = self.loss_metric()
        valid_score = self.score_metric()
        
        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                images = batch['X'].to(self.device)
                targets = batch['y'].to(self.device)
                
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                
                valid_loss.update(loss.detach().item())
                valid_score.update(targets, outputs)
                
            self.info_message(
                self.messages['batch'], 'Valid', step, len(valid_loader),
                valid_loss.avg, valid_score.avg, int(time.time() - t), end='\r'
            )
            
        return valid_loss.avg, valid_score.avg, int(time.time() - t)
    
    def save_model(self, num_epoch, save_path):
        torch.save(
            {
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'best_valid_score': self.best_valid_score,
                'num_epoch': num_epoch
            },
            save_path
        )
        
    @staticmethod
    def info_message(message, *args, end='\n'):
        print(message.format(*args), end=end)

In [ ]:
class LossMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0

    def update(self, val):
        self.n += 1
        self.avg = val / self.n + (self.n - 1) / self.n * self.avg

        
class AccMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0
        
    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().astype(int)
        y_pred = y_pred.cpu().numpy().argmax(axis=1).astype(int)
        last_n = self.n
        self.n += len(y_true)
        true_count = np.sum(y_true == y_pred)
        self.avg = true_count / self.n + last_n / self.n * self.avg

In [ ]:
train_fold = train_df[train_df["fold"] != Config.cfg['validation_fold']]
valid_fold = train_df[train_df["fold"] == Config.cfg['validation_fold']]

In [ ]:
train_set = CassavaDataset(df=train_fold, image_size=Config.cfg['image_size'], augments=Augments.train_augments)
valid_set = CassavaDataset(df=valid_fold, image_size=Config.cfg['image_size'], augments=Augments.val_augments)

In [ ]:
train_dataloader = DataLoader(
    train_set,
    batch_size=Config.cfg['batch_size'],
    shuffle=True,
    num_workers=Config.cfg['num_workers'],
)

valid_dataloader = DataLoader(
    valid_set,
    batch_size=Config.cfg['batch_size'],
    shuffle=False,
    num_workers=Config.cfg['num_workers']
)

In [ ]:
model = efficientnet_b0(Config.cfg['num_classes'])
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=Config.cfg['learning_rate'], weight_decay=Config.cfg['weight_decay'],)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=Config.cfg['warm_restarts_T_0'],
    eta_min=Config.cfg['warm_restarts_eta_min']
)

In [ ]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    loss_metric=LossMeter,
    score_metric=AccMeter
)

In [ ]:
history = trainer.fit(
    epochs=Config.cfg['epochs'],
    train_loader=train_dataloader,
    valid_loader=valid_dataloader,
    save_path=f'{Config.cfg["validation_fold"]}_fold_model_effnetb0_best.torch.',
    patience=Config.cfg['patience']
)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='train loss')
plt.plot(history['valid_loss'], label='valid loss')
plt.xticks(fontsize=14)
plt.xlabel("Epoch number", fontsize=15)
plt.yticks(fontsize=14)
plt.ylabel("Loss value", fontsize=15)
plt.legend(fontsize=15)
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(history['train_score'], label='train acc')
plt.plot(history['valid_score'], label='valid acc')
plt.xticks(fontsize=14)
plt.xlabel("Epoch number", fontsize=15)
plt.yticks(fontsize=14)
plt.ylabel("Accuracy score", fontsize=15)
plt.legend(fontsize=15)
plt.grid();

In [ ]:
if Config.neptune_settings["active"]:
    neptune_run.stop()